In this problem, you need to train a Hopfield network using the attached image. Use the attached training image to train your network and then evaluate its performance employing the attached test image. To this end, you need to complete each cell step by step.

You have to submit the final saved image as well as the completed notebook. 


In [37]:
import numpy as np
import random
import PIL
from PIL import Image
import os
import re
# from google.colab.patches import cv2_imshow 
# I ran code on my laptop. You can uncomment the above line and comment the below line to make it suitable to run on Colab
from cv2 import imshow as cv2_imshow 
import pickle

### My auxililiary functions for storing w instead of computing it every time. 

In [83]:
def pickle_writer(obj,path):
    with open(path, 'wb') as handle:
        pickle.dump(obj, handle, protocol=pickle.HIGHEST_PROTOCOL)

def pickle_reader(path):
    with open(path, 'rb') as handle:
        b = pickle.load(handle)
    return b

def w_calculator_saver(train_files="./train.jpg", path="./weights.pkl", threshold=145):
    train_image = readImg2array(train_files,size=(100,100), threshold= threshold)
    train_image_vec = mat2vec(train_image)
    w = create_W(train_image_vec)
    pickle_writer(w, path)

### Other functions

In [84]:
#Convert a matrix to a vector
def mat2vec(x):
    """convert the matrix x to a vector """
    tmp1 = x.flatten()
    return tmp1

In [85]:
def create_W(x):
    """  
    Create a square matrix with the same size as the input size.
    Note 1: The weight matrix must be symmetric
    Tip 1: For row i and column j, while i != j, place the value x [i] * x [j] in the weight matrix w [i, j]
    Tip 2: For row i and column j, while i=j, put the value 0 in the weight matrix w [i, j]
    """
    w = np.zeros((x.shape[0], x.shape[0]))
    for i in range(w.shape[0]):
        if i%1000==0:
            print(i)
        for j in range(i,w.shape[1]):
            if i!=j:
                w[i][j] = w[j][i] = x[i]*x[j]

    return w

In [86]:
#Read an image file and convert it to a pattern of the image
def readImg2array(file,size, threshold= 145):
    img = Image.open(file).convert(mode="L")
    img= img.resize(size)
    #img.thumbnail(size,Image.ANTIALIAS)
    imgArray = np.asarray(img,dtype=np.uint8)
    x = np.zeros(imgArray.shape,dtype=np.float)

    """
    Set the value to 1 for each pixel value with the larger than the threshold,
    and Set the value to -1 for each image pixel with a value of 0.
    """
    for i in range(x.shape[0]):
        for j in range(x.shape[1]):
            if imgArray[i][j] > threshold:
                x[i][j] = 1
            elif imgArray[i][j]==0:
                x[i][j] = -1

    return x

In [87]:
#Convert a numpy array to an image file like Jpeg
def array2img(data, outFile = None):

    #data is 1 or -1 matrix
    y = np.zeros(data.shape,dtype=np.uint8)
    y[data==1] = 255
    y[data==-1] = 0
    img = Image.fromarray(y,mode="L")
    if outFile is not None:
        img.save(outFile)
    return img

In [102]:
#Update the test input pattern (y_vec) based on the weight matrix
def update(w,y_vec,theta=0.5,time=100):

    """
    Once in a while, select a random number between 0 and the size of the input vector -1.
    Then use the random number line of the weight matrix to multiply internally by the input vector.
    Subtract the result from theta
    If the end result is greater than 0, enter a value of 1 in the input vector, otherwise replace -1.
    """
    for i in range(time):
        if i%10000==0:
            print(i)
        random_row = int(np.random.uniform(low=0.0, high=len(y_vec)-1, size=1)[0])
        product_result = np.dot(w[random_row,:],y_vec) - theta
        if product_result>0:
            y_vec[random_row]=1
        else:
            y_vec[random_row]=-1
        

    return y_vec

In [103]:
def hopfield(train_files, test_files,theta=0.5, time=1000, size=(100,100),threshold=60, load=False):
    
    """
    Using the built-in functions, Do the following steps:
    1- Read the input image and extract its pattern
    2. Convert the obtained pattern into a vector
    3- Make a weight matrix based on the vector of the previous step
    4- Read the test image and extract its pattern
    5- Convert the test pattern into a vector and give it as the input of the update function along with the built-in weight matrix.

    """
    
    train_image = readImg2array(train_files,size, threshold)
    train_image_vec = mat2vec(train_image)
    
    if not load:
        w = create_W(train_image_vec)
    else:
        w = pickle_reader("./weights.pkl")
        print("w loaded.")
    
    test_image = readImg2array(test_files,size, threshold)
    y_vec = mat2vec(test_image)

    y_vec_after = update(w=w,y_vec=y_vec,theta=theta,time=time)
    
    y_vec_after = y_vec_after.reshape(test_image.shape)
    
    after_img = array2img(y_vec_after,outFile=None)
    after_img.save("result.jpg")
    after_img.show()
    

In [104]:
threshold=60

In [91]:
# Run only once
w_calculator_saver(train_files="./train.jpg", path="./weights.pkl", threshold=threshold)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [110]:
# set load=True to load weights from saved pickle file instead of computing it every time for rapid testing of different parameters
hopfield("./train.jpg", "./test.jpg", theta=0.5,time=50000,size=(100,100),threshold=threshold, load=True)

w loaded.
0
10000
20000
30000
40000
